In [53]:
import pandas as pd
def get_direct_link(shared_link):
    return 'https://drive.google.com/uc?export=download&id=' + shared_link.split('/')[-2]

link = "https://drive.google.com/file/d/1sqMB2aRD2L77BsOiTDNjp-VzZX2NOEeR/view?usp=drive_link"

direct_link = get_direct_link(link)
combined_df = pd.read_csv(direct_link, encoding='utf-8')

players = pd.DataFrame(pd.concat([combined_df['Winner'],combined_df['Loser']]).unique())
players.columns = ['original_names']
players['search_query'] = players['original_names'] + " womens tennis wikipedia"

In [24]:
import requests
from bs4 import BeautifulSoup
import urllib.parse

def get_first_search_result_title(query):
    query = urllib.parse.quote_plus(query)
    url = f"https://www.google.com/search?q={query}"

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
        'Accept-Language': 'en-US,en;q=0.5'
    }
    
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")
    first_result = soup.find("div", {"class": "yuRUbf"})
    if first_result:
        title = first_result.find("h3").text
        return title.strip()

    return None

# due to limits on requests we use the count value to show which names have been searched
count = 0
for index, row in players.iterrows():
    if index < count:
        continue
    query = row["search_query"]
    result_title = get_first_search_result_title(query)
    if not result_title:
        print(count,index,row['original_names'])
        break
    players.at[index, "wiki_title"] = result_title if result_title else "No search results found"

In [29]:
players["wiki_title"] = players["wiki_title"].str.replace(' - Wikipedia', '')

In [32]:
corrections = {
    'Radwanskax U.': 'Urszula Radwańska',
    'Zheng S.': 'Zheng Saisai',
    'Knapp K.': 'Karin Knapp',
    'Allertova D.': 'Denisa Šátralová',
    'Chang K.C.': 'Chang Kai-chen',
    'Ce G.': 'Gabriela Cé',
    'Zhang K-L.': 'Zhang Kailin',
    'Zhang K.L.': 'Zhang Kailin',
    'King V.': 'Vania King',
    'Zhang Yux.': 'Zhang Yuxuan',
    'Zhang Yi.': 'Zhang Ying (tennis)',
    'Liu F.': 'Liu Fangzhou',
    'Krejsova P.': 'Petra Krejsová',
    'Wang Y.': 'Wang Yafan',
    'Sanchez A.S.': 'Ana Sofía Sánchez',
    'Mayr P.': 'Patricia Mayr-Achleitner',
    'Perez Garcia M.P.': 'María Paulina Pérez',
    'Safarova Z.': 'Zuleykha Safarova',
    'Zhang L.': 'Zhang Ling (tennis)',
    'Shoshyna A.': 'Anastasiya Shoshyna',
    'Pous-T L.': 'Laura Pous Tió',
    'Xu S.': 'Xu Shilin',
    'Zheng W.': 'Zheng Wushuang',
    'Ng K.Y.': 'Ng Kwan-yau',
    'Lee S.R.': 'Lee So-ra (tennis)',
    'Park S.': 'Park So-hyun (tennis)',
    'Bai A.': 'Alison Bai',
    'Wang M.': 'Wang Meiling',
    'Ma S.': 'Ma Shuyue',
    'Bogdan E.': 'Elena Bogdan',  
    'Rosca A.': 'Andreea Roșca',
    'Alves M.F.': 'Maria Fernanda Alves',
    'Patterson T.': 'Tammi Patterson',
    'Lee P.C.': 'Lee Pei-chi',
    'Mayo A.': 'Alyssa Mayo',
    'Choi J-H.': 'Choi Ji-hee',
    'Fruhvirtova B.': 'Brenda Fruhvirtová',
    'Yu E.': 'Eleana Yu',
    'Back D.': 'Back Da-yeon',
    'Crawley F.': 'Fiona Crawley',
}

In [33]:
players['wiki_title'] = players['original_names'].map(corrections).fillna(players['wiki_title'])

In [34]:
import numpy as np
no_pages = ['Benhadi G.', 'El Aouni A.', 'Ewald W.']
no_pages_idx = [419, 728, 744]
for p in no_pages_idx:
    players.loc[p,'wiki_title'] = np.nan

In [36]:
players.to_csv("wiki_players.csv", encoding='utf-8', index=False)